In [1]:
import os
import time
import json
import random
from pathlib import Path
from functools import partial
from pprint import pprint

from urllib.parse import urlencode, urlparse, urlunparse, parse_qs, unquote

import pandas as pd
import numpy as np

from omegaconf import OmegaConf

from transformations.id import IDRules

import ipyplot as iplt
import matplotlib.pyplot as plt

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

%load_ext autoreload
%autoreload 2

In [2]:
merchant = "adidas"
cfg = OmegaConf.load(f"conf/merchants/{merchant}.yaml")
OmegaConf.to_container(cfg)

{'merchant': 'adidas',
 'feed': {'url': 'https://feeds.performancehorizon.com/aisl3/1101l532/791b5c4afe8b4136acd51c89e28a5ed4.csv',
  'compression': None},
 'id': None,
 'variant': 'mpn',
 'codes': {'mpn': 'style',
  'product_GTIN': 'gtin',
  'merchant_product_id': 'ac'},
 'column_map': {'brand_name': 'brand',
  'product_name': 'product_name',
  'merchant_product_id': 'merchant_sku',
  'description': 'description',
  'product_short_description': 'short_description',
  'custom_3': 'category',
  'merchant_deep_link': 'merchant_deep_link',
  'custom_1': 'age',
  'Fashion:suitable_for': 'gender',
  'Fashion:size': 'size',
  'Fashion:material': 'material',
  'display_price': 'price',
  'merchant_image_url': 'image_url',
  'colour': 'color'},
 'filters': [{'col': 'category_name',
   'include': ["Women's Footwear", "Men's Footwear"]},
  {'col': 'custom_1', 'include': 'adult'},
  {'col': 'custom_3', 'exclude': 'Accessories'}],
 'groupby': None,
 'code': ['mpn', 'merchant_product_id']}

In [3]:
df = pd.read_csv(cfg.feed.url, compression=cfg.feed.compression, low_memory=False)
df


,Product Name,Product Image URL,Product Value,Product Category,Product ID,Product URL,mpn,upc,brand,color,currency,old_price,shipping_cost,quantity,size,product_hierarchy,EAN,gender,Description,sport,style,additional_image_1,additional_image_2,additional_image_3,additional_image_4,specifications,material,delivery_time
0,Condivo 22 Sweat Tracksuit Bottoms,"https://assets.adidas.com/images/w_1080,h_1080...",55.0,"Clothing&amp;Trousers,Joggers,Tracksuit Bottoms",HD2298-0005,https://adidas.prf.hn/click/camref:1101leh8N/c...,F6540,"195733310943,195733310820,195733306830,1957333...",adidas,Team Grey Four,GBP,55.0,0.00,69,XL,variant,4.065425e+12,Men,Created for top level football. Part of Condiv...,Football,SPORTSWEAR,"https://assets.adidas.com/images/w_1080,h_1080...","https://assets.adidas.com/images/w_1080,h_1080...","https://assets.adidas.com/images/w_1080,h_1080...","https://assets.adidas.com/images/w_1080,h_1080...",Regular fit with mid rise|Elastic waist with d...,Cotton,2-5 days
1,Condivo 22 Sweat Tracksuit Bottoms,"https://assets.adidas.com/images/w_1080,h_1080...",55.0,"Clothing&amp;Trousers,Joggers,Tracksuit Bottoms",HD2298-0006,https://adidas.prf.hn/click/camref:1101leh8N/c...,F6540,"195733310943,195733310820,195733306830,1957333...",adidas,Team Grey Four,GBP,55.0,0.00,22,2XL,variant,4.065425e+12,Men,Created for top level football. Part of Condiv...,Football,SPORTSWEAR,"https://assets.adidas.com/images/w_1080,h_1080...","https://assets.adidas.com/images/w_1080,h_1080...","https://assets.adidas.com/images/w_1080,h_1080...","https://assets.adidas.com/images/w_1080,h_1080...",Regular fit with mid rise|Elastic waist with d...,Cotton,2-5 days
2,Condivo 22 Hoodie,"https://assets.adidas.com/images/w_1080,h_1080...",60.0,Clothing&amp;Hoodies,HD2306-0001,https://adidas.prf.hn/click/camref:1101leh8N/c...,WR413,"195733264550,195733268442,195733264581,1957332...",adidas,Team Grey Four,GBP,60.0,0.00,33,XS,variant,4.065425e+12,Men,Wear the football look all day. This adidas Co...,Football,SPORTSWEAR,"https://assets.adidas.com/images/w_1080,h_1080...","https://assets.adidas.com/images/w_1080,h_1080...","https://assets.adidas.com/images/w_1080,h_1080...","https://assets.adidas.com/images/w_1080,h_1080...",Regular fit|Drawcord-adjustable hood|Main: 53%...,Cotton,2-5 days
3,Condivo 22 Hoodie,"https://assets.adidas.com/images/w_1080,h_1080...",60.0,Clothing&amp;Hoodies,HD2306-0002,https://adidas.prf.hn/click/camref:1101leh8N/c...,WR413,"195733264550,195733268442,195733264581,1957332...",adidas,Team Grey Four,GBP,60.0,0.00,98,S,variant,4.065425e+12,Men,Wear the football look all day. This adidas Co...,Football,SPORTSWEAR,"https://assets.adidas.com/images/w_1080,h_1080...","https://assets.adidas.com/images/w_1080,h_1080...","https://assets.adidas.com/images/w_1080,h_1080...","https://assets.adidas.com/images/w_1080,h_1080...",Regular fit|Drawcord-adjustable hood|Main: 53%...,Cotton,2-5 days
4,Condivo 22 Hoodie,"https://assets.adidas.com/images/w_1080,h_1080...",60.0,Clothing&amp;Hoodies,HD2306-0003,https://adidas.prf.hn/click/camref:1101leh8N/c...,WR413,"195733264550,195733268442,195733264581,1957332...",adidas,Team Grey Four,GBP,60.0,0.00,188,M,variant,4.065425e+12,Men,Wear the football look all day. This adidas Co...,Football,SPORTSWEAR,"https://assets.adidas.com/images/w_1080,h_1080...","https://assets.adidas.com/images/w_1080,h_1080...","https://assets.adidas.com/images/w_1080,h_1080...","https://assets.adidas.com/images/w_1080,h_1080...",Regular fit|Drawcord-adjustable hood|Main: 53%...,Cotton,2-5 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81257,Runfalcon 2.0 Shoes,"https://assets.adidas.com/images/w_1080,h_1080...",28.0,"Shoes&amp;Trainers,Gym Trainers,Lifestyle Trai...",GX3530-0003,https://adidas.prf.hn/click/camref:1101leh8N/c...,LUT59,"195734732737,195734732782,195734732775,1957347...",adidas,Core Black / Cloud White / Silver Metallic,GBP,28.0,1.99,21,11K,v

In [4]:
df.columns.tolist()

['Product Name',
 'Product Image URL',
 'Product Value',
 'Product Category',
 'Product ID',
 'Product URL',
 'mpn',
 'upc',
 'brand',
 'color',
 'currency',
 'old_price',
 'shipping_cost',
 'quantity',
 'size',
 'product_hierarchy',
 'EAN',
 'gender',
 'Description',
 'sport',
 'style',
 'additional_image_1',
 'additional_image_2',
 'additional_image_3',
 'additional_image_4',
 'specifications',
 'material',
 'delivery_time']

In [5]:
for col in df.columns:
    print("-" * 100)
    print(col)
    print("-" * 100)
    print(df[col].value_counts().head(20))

----------------------------------------------------------------------------------------------------
Product Name
----------------------------------------------------------------------------------------------------
Superstar Shoes                  977
Stan Smith Shoes                 765
Ultraboost 22 Shoes              468
OZWEEGO Shoes                    373
Forum Low Shoes                  360
Runfalcon 2.0 Shoes              331
Gazelle Shoes                    314
Ozelia Shoes                     306
ZX 22 BOOST Shoes                269
Swift Run 22 Shoes               258
Astir Shoes                      255
Ultraboost 5.0 DNA Shoes         236
Superstar 360 Shoes              223
adidas Racer TR x LEGO® Shoes    219
NMD_V3 Shoes                     212
Duramo 10 Shoes                  206
Run Falcon 2.0 Shoes             196
NMD_R1 Shoes                     196
Forum Bold Shoes                 195
Adilette Aqua Slides             178
Name: Product Name, dtype: int64
------------

In [6]:
df.columns

Index(['Product Name', 'Product Image URL', 'Product Value',
       'Product Category', 'Product ID', 'Product URL', 'mpn', 'upc', 'brand',
       'color', 'currency', 'old_price', 'shipping_cost', 'quantity', 'size',
       'product_hierarchy', 'EAN', 'gender', 'Description', 'sport', 'style',
       'additional_image_1', 'additional_image_2', 'additional_image_3',
       'additional_image_4', 'specifications', 'material', 'delivery_time'],
      dtype='object')

In [22]:
filter_col = df["Product ID"].str.split("-").str[0]
# filter_col = df["mpn"] + "_" + df["color"]
# filter_col = df["Product Name"]
# frame = df[filter_col == random.choice(filter_col)].drop_duplicates("Product Image URL")
frame = df[filter_col == "SB3757123896"]
iplt.plot_images(
    frame["Product Image URL"].values,
    labels=frame["Product Name"].values,
    custom_texts=frame[["Product ID", "mpn", "color", "size"]].apply(" <HR> ".join, axis=1).values,
    show_url=False,
    max_images=50
)



In [107]:
x = df.assign(mpn_color=df["mpn"] + "_" + df["color"], id_split=df["Product ID"].str.split("-").str[0]).groupby("id_split").mpn_color.agg("unique") x[x.apply(len) > 1]

Series([], Name: mpn_color, dtype: object)

In [98]:
df["Product ID"].str.split("-").str[0].nunique()

14784

In [22]:
id_rule = IDRules(config, merchant)
df = df.dropna(subset=[config.pid[merchant]]).reset_index(drop=True)
df["id"] = df.apply(id_rule, axis=1)
df = df.dropna(subset=["id"]).reset_index(drop=True)
df

,aw_deep_link,product_name,aw_product_id,merchant_product_id,merchant_image_url,description,merchant_category,search_price,merchant_name,merchant_id,category_name,category_id,aw_image_url,currency,store_price,delivery_cost,merchant_deep_link,language,last_updated,display_price,data_feed_id,brand_name,brand_id,colour,product_short_description,specifications,condition,product_model,model_number,dimensions,keywords,promotional_text,product_type,commission_group,merchant_product_category_path,merchant_product_second_category,merchant_product_third_category,rrp_price,saving,savings_percent,base_price,base_price_amount,base_price_text,product_price_old,delivery_restrictions,delivery_weight,warranty,terms_of_contract,delivery_time,in_stock,stock_quantity,valid_from,valid_to,is_for_sale,web_offer,pre_order,stock_status,size_stock_status,size_stock_amount,merchant_thumb_url,large_image,alternate_image,aw_thumb_url,alternate_image_two,alternate_image_three,alternate_image_four,reviews,average_rating,rating,number_available,custom_1,custom_2,custom_3,custom_4,custom_5,custom_6,custom_7,custom_8,custom_9,ean,isbn,upc,mpn,parent_product_id,product_GTIN,basket_link,Fashion:suitable_for,Fashion:category,Fashion:size,Fashion:material,Fashion:pattern,Fashion:swatch,id
0,https://www.awin1.com/pclick.php?p=25414669965...,By Terry Terrybly Velvet Rouge Lipstick 2ml (V...,25414669965,11307221,https://s4.thcdn.com/productimg/1600/1600/1130...,Reveal a satin-finished smile with the By Terr...,Cosmetics,32.0,Coggles UK,4318,Cosmetics,111,https://images2.productserve.com/?w=200&h=200&...,GBP,NaN,4.99,https://www.coggles.com/by-terry-terrybly-velv...,NaN,NaN,GBP32.00,4318,By Terry,0,7. Bankable Rose,Reveal a satin-finished smile with the By Terr...,NaN,new,NaN,NaN,NaN,NaN,NaN,NaN,EXISTING_CUSTOMER,Health and Beauty > Cosmetics > NULL,NaN,NaN,32.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 days,1,NaN,NaN,NaN,1,0,0,NaN,NaN,NaN,https://s4.thcdn.com/productimg/70/70/11307221...,NaN,NaN,https://images2.productserve.com/?w=70&h=70&bg...,NaN,NaN,NaN,0,NaN,NaN,NaN,adult,30-03-2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.700076e+12,3.700076e+12,3.700076e+12,1141581700,11307214.0,3.700076e+12,https://www.coggles.com/my.basket?buy=11307221...,unisex,NaN,NaN,NaN,NaN,NaN,coggles.113072140
1,https://www.awin1.com/pclick.php?p=22949291513...,By Terry Hyaluronic Sheer Rouge Lipstick 3g (V...,22949291513,11307103,https://s4.thcdn.com/productimg/1600/1600/1130...,Achieve smooth and youthful lips with the By T...,Cosmetics,29.0,Coggles UK,4318,Cosmetics,111,https://images2.productserve.com/?w=200&h=200&...,GBP,NaN,4.99,https://www.coggles.com/by-terry-hyaluronic-sh...,NaN,NaN,GBP29.00,4318,By Terry,0,12. Be Red,Achieve smooth and youthful lips with the By T...,NaN,new,NaN,NaN,NaN,NaN,NaN,NaN,EXISTING_CUSTOMER,Health and Beauty > Cosmetics > NULL,NaN,NaN,29.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 days,1,NaN,NaN,NaN,1,0,0,NaN,NaN,NaN,https://s4.thcdn.com/productimg/70/70/11307103...,NaN,https://s4.thcdn.com//productimg/1600/1600/113...,https://images2.productserve.com/?w=70&h=70&bg...,NaN,NaN,NaN,0,NaN,NaN,NaN,adult,02-10-2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.700076e+12,3.700076e+12,3.700076e+12,1141601200,11307091.0,3.700076e+12,https://www.coggles.com/my.basket?buy=11307103...,unisex,NaN,NaN,NaN,NaN,NaN,coggles.113070910
2,https://www.awin1.com/pclick.php?p=22949291487...,By Terry Moisturising CC Cream 30ml (Various S...,22949291487,11307059,https://s4.thcdn.com/productimg/1600/1600/1130...,"Correct, perfect and protect your complexion w...",Cosmetics,58.0,Coggles UK,4318,Cosmetics,111,https://images2.productserve.com/?w=200&h=200&...,GBP,NaN,4.99,https://www.coggles.com/by-terry-moisturising-...,NaN,NaN,GBP58.00,4318,By Terry,0,4. Tan,"Correct, perfect and protect your complexion w...",Shake before use. Apply with the By Terry numb...,new,NaN,NaN,NaN,NaN,NaN,NaN,EXISTING_CUSTOMER,Health and Beauty > Cosmetics > NULL,NaN,NaN,58.0,0.0,0.0,58.0,40.0,g,NaN,NaN,NaN,NaN,NaN,3 days,1,Na

In [23]:
df[config.pid[merchant]].value_counts().sum() == df.id.value_counts().sum()

True

In [24]:
df.id.value_counts()

coggles.113874640    32
coggles.113786830    29
coggles.112646250    26
coggles.123206080    25
coggles.114692970    16
                     ..
coggles.134418060     1
coggles.130539730     1
coggles.130349500     1
coggles.134573470     1
coggles.133268320     1
Name: id, Length: 5404, dtype: int64

In [25]:
df.merchant_category.value_counts().head(100)

Clothing                               10160
Footwear                                3761
Cosmetics                                420
Children's Footwear                      317
Jackets [Clothing/Men/Clothing]          277
Trainers [Clothing/Men/Footwear]         235
Boots [Clothing/Women/Footwear]          196
T-Shirts [Clothing/Men/Clothing]         192
Sweatshirts [Clothing/Men/Clothing]      153
Trainers [Clothing/Women/Footwear]       150
Sandals [Women / Footwear]               147
Children's Clothing                      127
Sports Clothing                          126
Mens - Polo Shirts                       116
Sports - Trousers                         84
Sports - Vests                            82
Coats [Clothing/Women/Clothing]           81
T-Shirts & Tops [Clthng/W/Clthng]         79
Flats [Women / Footwear]                  73
Tops [Clothing/Men/Clothing]              72
Womens - Accessories - Jewellery          71
Boots [Clothing/Men/Footwear]             70
Womens - N

In [26]:
df.merchant_product_category_path.value_counts().head(30)

Apparel > Clothing                          10013
Apparel > Footwear                           3819
Apparel > Footwear > Trainers                 479
Apparel > Clothing > T-Shirts & Tops          456
Health and Beauty > Cosmetics > NULL          416
Apparel > Footwear > Children's Footwear      317
Apparel > Clothing > Jackets                  313
Apparel > Footwear > Boots                    267
Apparel > Clothing > Sweatshirts              242
Apparel > Clothing > Sports Apparel           214
Apparel > Footwear > Sandals                  194
Apparel > Clothing > Children's Clothing      158
Apparel > Clothing > Polo Shirts              124
Apparel > Clothing > Trousers                 119
Apparel > Footwear > Shoes                    109
Apparel > Clothing > Shorts                    98
Apparel > Clothing > Underwear                 86
Apparel > Clothing > Coats                     84
Apparel > Footwear > Flats                     73
Apparel > Accessories > Jewellery              71


In [27]:
(data_dir / "filtered").mkdir(exist_ok=True)

def filter_categories(df, category_col, category_vals):
    print("category col:", category_col)
    print(df[category_col].value_counts().head(20))
    print("category vals:")
    print(category_vals)
    print()
    print("rows before: ", len(df))
    df = df.loc[df[category_col].isin(set(category_vals))].reset_index(drop=True)
    print("rows after: ", len(df))
    print()
    print("cols retained:")
    print(df[category_col].value_counts())
    return df

df = filter_categories(df, config.filter[merchant].col, config.filter[merchant].vals)
if "col2" in config.filter[merchant]:
    df = filter_categories(df, config.filter[merchant].col2, config.filter[merchant].vals2)
df.to_parquet(data_dir / "filtered" / f"{merchant}.parquet")

category col: category_name
Clothing Accessories        10212
Shoes                        3853
Women's Footwear              711
Men's Tops                    578
Cosmetics                     448
Men's Footwear                425
Women's Outerwear             358
Children's Footwear           327
Sportswear & Swimwear         295
Women's Tops                  132
Children's Clothing           127
General Clothing              126
Men's Trousers                 87
Men's Accessories              77
Men's Clothing                 75
Nightwear                      72
Other Sports                   72
Women's Jewellery              71
Men's Underwear                71
Women's Dresses & Skirts       58
Name: category_name, dtype: int64
category vals:
['Shoes', 'Boots', "Women's Footwear", "Men's Footwear"]

rows before:  18454
rows after:  4989

cols retained:
Shoes               3853
Women's Footwear     711
Men's Footwear       425
Name: category_name, dtype: int64


In [28]:
df.id.value_counts()

coggles.127115520    13
coggles.136300170    10
coggles.117880490     9
coggles.125962000     9
coggles.117234470     9
                     ..
coggles.131237300     1
coggles.127918330     1
coggles.120299530     1
coggles.131238270     1
coggles.129453990     1
Name: id, Length: 1235, dtype: int64

In [29]:
df.id.nunique()

1235

In [15]:
df.brand_name.value_counts().head(40)

AttributeError: 'DataFrame' object has no attribute 'brand_name'

In [16]:
df.colour.value_counts().head(50)

AttributeError: 'DataFrame' object has no attribute 'colour'

In [ ]:
for col in df.columns:
    print(col)
    print("-" * 100)
    print(df[col].value_counts().head(20))
    print("-" * 100)

In [ ]:
df.groupby("parent_product_id").product_name.value_counts().head(100)

In [ ]:
random.choices(df.DESCRIPTION.values, k=5)

In [ ]:
random.choices(df.product_short_description.values, k=5)